In [12]:
from langdetect import detect
import json

In [30]:
def normalize_rate(r):
    if r > 3:
        return 1
    elif r == 3:
        return 0
    else:
        return -1

# TODO: include adv and shortcomings
# TODO: include votes
# TODO: clean html tags from text
def build_corpus(f):
    X, y = [], []
    
    with open(f, 'r') as rows:
        try:
            for row in rows:
                comment = json.loads(row)
                
                if len(comment['text']) > 2:
                    if comment['text'] and detect(comment['text']) == 'uk':
                        X.append(comment['text'])
                        y.append(normalize_rate(comment['rate'] or 0))
                else:
                    continue
        except:
            print(comment)

    return X, y

In [58]:
X, y = build_corpus('comments.json')

{'id': 15249222, 'name': 'Дмитрий', 'rate': 5, 'text': '...', 'advantages': 'качество работы, тихий, дизайн', 'shortcomings': 'возможно цена, небольшей бак для воды.', 'positive_votes_count': 1, 'negative_votes_count': 0}


In [59]:
print(len(X), len(y))

349 349


In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1789)

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(X_train)

<261x2432 sparse matrix of type '<class 'numpy.int64'>'
	with 5435 stored elements in Compressed Sparse Row format>

In [62]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(vectorizer.transform(X_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [65]:
from sklearn.metrics import classification_report

X_features_test = vectorizer.transform(X_test)
y_pred = clf.predict(X_features_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.71      0.40      0.51        43
           0       0.00      0.00      0.00         1
           1       0.58      0.84      0.69        44

    accuracy                           0.61        88
   macro avg       0.43      0.41      0.40        88
weighted avg       0.64      0.61      0.59        88

